In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [5]:
appearances_train = pd.read_csv("appearances_first_3_days.csv", header=None, names=["device_id", "date", "seconds_until_next_appearence"])

In [6]:
appearances_test =  pd.read_csv("appearances_second_3_days.csv", header=None, names=["device_id", "date", "seconds_until_next_appearence"])

In [7]:
appearances_train["date"] = pd.to_datetime(appearances_train["date"])

In [8]:
appearances_test["date"] = pd.to_datetime(appearances_test["date"])

In [9]:
#agrego feature si es fin de semana o no

In [10]:
appearances_train["first_appearence_is_weekend"] = appearances_train.apply(lambda row: row["date"].weekday()>4 , axis=1)

In [11]:
appearances_test["first_appearence_is_weekend"] = appearances_test.apply(lambda row: row["date"].weekday()>4 , axis=1)

In [12]:
events = pd.read_csv("events.csv")

/home/martinrosas/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
installs = pd.read_csv("installs.csv")

/home/martinrosas/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
#creo feature -> para cada id cual es la app mas usada por cada dispositivo a partir del dataframe EVENTS

In [15]:
most_used_app = events.groupby("ref_hash")["application_id"].agg(lambda x: x.mode()).rename_axis("device_id").to_frame().reset_index()


In [16]:
#arreglo que algunos casos la app id mas usada es una lista. porque la funcion .mode() devolvio mas de un elemento
most_used_app["application_id"] = most_used_app["application_id"].apply(lambda x: x[0] if type(x) == np.ndarray else x)

In [17]:
df_train = appearances_train.merge(most_used_app, on="device_id", how="left")

In [18]:
df_test = appearances_test.merge(most_used_app, on="device_id", how="left")

In [19]:
#renombro para dejar mas claro que el nombre del nuevo feature
df_train = df_train.rename({"application_id": "most_used_app"}, axis="columns")
df_test = df_test.rename({"application_id": "most_used_app"}, axis="columns")

In [20]:
#reemplazo los nan de most used app por un nuemero no existente
df_train["most_used_app"] = df_train["most_used_app"].fillna(-1)
df_test["most_used_app"] = df_test["most_used_app"].fillna(-1)

In [21]:
def is_android_or_macos(row):
    if row["user_agent"] is np.nan:
        return None
    agent = row["user_agent"].lower()
    if "android" in agent:
        return "android"
    elif "darwin" in agent:
        return "mac os"
    else:
        return "other"

In [22]:
installs["os"] = installs.apply(is_android_or_macos, axis=1)

In [23]:
os = installs[["ref_hash", "os"]].dropna()

In [24]:
#saco ids duplicados
os = os.drop_duplicates(subset="ref_hash", keep='first').rename({"ref_hash": "device_id"},axis="columns")

In [25]:
#lo agrego a DF, siempre left join
df_train = df_train.merge(os,how="left", on="device_id")
df_test = df_test.merge(os,how="left", on="device_id")

In [26]:
df_train["os"] = df_train["os"].fillna("other")
df_test["os"] = df_test["os"].fillna("other")

In [27]:
#PRUEBO OTRA VEZ con random forest
#antes tengo que convertir a numeros los campos que son string (en este caso solo OS)
#USO LABEL ENCODER
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [28]:
#encodeo columna os
le.fit(df_train["os"])
df_train["os"] = le.transform(df_train["os"])
df_test["os"] = le.transform(df_test["os"])

In [29]:
#ahora en vez de string tengo sus codigos
df_train["os"].head()

0    2
1    2
2    2
3    2
4    2
Name: os, dtype: int64

In [30]:

#CREO X_TRAIN Y Y_TRAIN SIN DEVICE ID
X_train = df_train[['first_appearence_is_weekend',
       'most_used_app', 'os']]
y_train = df_train['seconds_until_next_appearence']
X_test = df_test[['first_appearence_is_weekend',
       'most_used_app', 'os']]
y_test = df_test['seconds_until_next_appearence']

In [31]:
clf = RandomForestRegressor()
#entreno el modeo
clf.fit(X_train, y_train)  
y_pred = clf.predict(X_test)

/home/martinrosas/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [32]:
#en y_pred tengo las predicciones y en y_test tengo los valores de verdad que queria predecir
#mido el error

print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))

Mean squared error: 134283789.22


In [33]:
#ahora agrego como feature el device ID y mido el error a ver que onda

In [34]:

#CREO X_TRAIN Y Y_TRAIN CON DEVICE ID
X_train = df_train[['first_appearence_is_weekend',
       'most_used_app', 'os', 'device_id']]
y_train = df_train['seconds_until_next_appearence']
X_test = df_test[['first_appearence_is_weekend',
       'most_used_app', 'os', 'device_id']]
y_test = df_test['seconds_until_next_appearence']

In [35]:
clf = RandomForestRegressor()
#entreno el modeo
clf.fit(X_train, y_train)  
y_pred = clf.predict(X_test)

/home/martinrosas/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [36]:
#en y_pred tengo las predicciones y en y_test tengo los valores de verdad que queria predecir
#mido el error

print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))

Mean squared error: 393557254.59


In [37]:
#EL ERROR PASO DE 134,284,950 A 393,557,254
#parece que no conviene usar el ID como feature. pruebo mas adelante cuando tengamos mas features

In [ ]:
#busco mas features